In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
df = pd.read_csv('modified_teleco_customer_churn.csv')
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,29.85,0
1,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1889.50,0
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,108.15,1
3,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1840.75,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,151.65,1


In [3]:
X = df.drop('Churn',axis=1)
y = df['Churn']

In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state=42)

In [5]:
param_grids = {
    "Logistic Regression": (LogisticRegression(max_iter=1000), {
        'C': [1, 10],
        'solver': ['liblinear', 'lbfgs']
    }),
    "Decision Tree": (DecisionTreeClassifier(), {
        'max_depth': [3, 5, None],
        'criterion': ['gini', 'entropy']
    }),
    "Random Forest": (RandomForestClassifier(), {
        'n_estimators': [50, 100],
        'max_depth': [5, None]
    }),
    "KNN": (KNeighborsClassifier(), {
        'n_neighbors': [5, 7]
    }),
    "Naive Bayes": (GaussianNB(), {})  # No hyperparameters to tune
}

In [6]:
best_model = None
best_accuracy = 0
best_params = None

In [7]:
for name, (model, params) in param_grids.items():
    if params:
        grid = GridSearchCV(model, params, cv=5, scoring='accuracy')
        grid.fit(X_train, y_train)
        model = grid.best_estimator_
        print(f"{name} Best Params: {grid.best_params_}")
    else:
        model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        best_params = grid.best_params_ if params else "Default Parameters"

print(f"Best model: {best_model}")
print(f"Best params: {best_params}")
print(f"Best accuracy: {best_accuracy}")

C:\Users\heral\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\heral\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stab

Logistic Regression Best Params: {'C': 1, 'solver': 'liblinear'}
Logistic Regression Accuracy: 0.8161816891412349
              precision    recall  f1-score   support

           0       0.86      0.90      0.88      1036
           1       0.68      0.58      0.63       373

    accuracy                           0.82      1409
   macro avg       0.77      0.74      0.75      1409
weighted avg       0.81      0.82      0.81      1409

[[933 103]
 [156 217]]
Decision Tree Best Params: {'criterion': 'gini', 'max_depth': 5}
Decision Tree Accuracy: 0.794180269694819
              precision    recall  f1-score   support

           0       0.87      0.85      0.86      1036
           1       0.61      0.64      0.62       373

    accuracy                           0.79      1409
   macro avg       0.74      0.74      0.74      1409
weighted avg       0.80      0.79      0.80      1409

[[881 155]
 [135 238]]
Random Forest Best Params: {'max_depth': 5, 'n_estimators': 50}
Random Forest A

In [8]:
pickle.dump(best_model, open("model.pkl", 'wb'))